##### Laboratoire 2
# Discrimination linéaire et réseaux de neurones profonds
## Classification et régression (Datasets FER et FG-NET)

### GTI771 - Apprentissage machine avancé
#### Département de génie logiciel et des technologies de l’information (LogTI)

##### <font color=white> Version 1.0 - février 2020<br>Version 2.0 - février 2020 <br> Version 3.0 - mars 2020<br>Version 4.0 - juin 2020 (Richard Rail)<br>Version 4.1 - février 2021<br></font><br>Version 4.2 - février 2022<br>


##### Prof. Alessandro L. Koerich

| Étudiants             | Antoine Brassard Lahey - BRAA05089904 <br/>  Antoine Pelchat-Fortin - PELA04029703 <br/> Sébastien Charbonneau - CHAS01049805               |
|-----------------------|---------------------------------------------------------|
| Session               | ÉTÉ 2022                                                |
| Équipe                | 10                                                      |
| Numéro du laboratoire | 2                                                       |
| Professeur            | Hervé Lombaert                                          |
| Chargé de laboratoire | Lucas Geffard                                           |
| Date                  | 2022-07-12                                              |

# Introduction

Ce deuxième laboratoire porte sur l'utilisation de trois algorithmes d'apprentissage soit les algorithmes de régression, les réseaux neuronaux et les réseaux neuronaux profonds. Dans ce laboratoire, vous êtes amenés à utiliser de nouvelles approches à l’aide de ces algorithmes aﬁn de résoudre deux problèmes: prédiction de l'âge de personnes à partir de photos du visage (régression); problème de classiﬁcation des expressions faciales (FER) introduit dans le cadre du premier laboratoire de ce cours.

Le problème de régression qui vous est présenté est le problème [Facial Aging Estimation (FAE)](https://yanweifu.github.io/FG_NET_data/index.html), dont le but est de prédire l'âge des personnes à partir du visage. En vous basant sur les concepts vus en classe et l'expérience acquise dans le laboratoire 1, vous êtes invité à reprendre les primitives développées lors du laboratoire 1 ou d'autres primitives que vous jugez pertinentes à extraire sur ces types d’images et effectuer l’extraction de celles-ci sur l’ensemble de données fournies avec cet énoncé. 

##### Description de l'ensemble de données:
* 1002 images faciales de 82 sujets multiraciaux âgés de 0 à 69 ans;
* Déséquilibré: 50% des sujets ont entre 0 et 13 ans;
* Images couleur et niveaux de gris avec une dimension moyenne de 384x487 pixels, et la résolution varie de 200 dpi à 1200 dpi;
* Grande variation d'éclairage, de pose, d'expression faciale, de flou et d'occlusions (par exemple, moustache, barbe, lunettes, etc.).

Voici, en exemple, des images de visages se retrouvant dans l’ensemble de données FG-NET:

![Exemples de FER](https://www.mdpi.com/sensors/sensors-16-00994/article_deploy/html/images/sensors-16-00994-g001.png)

Veuillez noter que les images qui vous sont fournies ne sont pas nécessairement similaires aux images de FER. Plusieurs images comportent du bruit, des artéfacts ou des éléments non pertinents. Le défi de ce laboratoire repose sur cette difficulté qui est chose courante dans des problèmes d’apprentissage machine moderne.

Tout comme le premier travail pratique, vous réaliserez ce deuxième laboratoire avec la technologie Python3 conjointement avec la librairie d’apprentissage machine scikit-learn et TensorFlow et Keras pour la partie réseaux de neurones. Vous êtes invité à reprendre le code développé lors du laboratoire 1 afin de continuer son développement.

<font color=black> L’évaluation de ce laboratoire sera basée sur la qualité des modèles entraînés, la comparaison des performances réalisées par les différents modèles, les réponses aux questions dans cette notebook ainsi que l'organisation de votre code source (SVP, n'oubliez pas des commentaires dans le code!).</font>

* #### Partie 1: Régression
* Régression lineaire
* Régression Ridge
* Régression Lasso et Elastic-Net
* Descente du gradiente stochastique (SGD)<br>
<br>
* #### Partie 2: Classification
* Régression logistique
* Réseaux de neurones MLP <br>
<br>
* #### Partie 3: Classification et régression
* Réseaux convolutionel entraîné "from scratch"
* Réseaux convolutionel + modèles pre-entraînes (transfer learning)
* Réseaux convolutionel adapté à la régression

## Partie 0: Imports

#### (0a) Import de bibliotèques



##### À faire:
1. Ajouter toutes les bibliothèques que vous avez utilisées pour compléter ce notebook dans une cellule avec une petite description.

In [ ]:
# Numpy - Used to manipulate matrices
import numpy as np

# Matplotlib - 2D plotting library
import matplotlib.pyplot as plt 
from matplotlib.pyplot import figure

# Library to extract features from images. Used for LBP
import skimage
from skimage import feature
from skimage.restoration import denoise_tv_chambolle

# SKLearn - Implementations of different machine learning models, used for tuning as well
import absl
import sklearn
import sklearn.metrics
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif

import tensorflow
import tensorflow.keras as keras
from tensorflow.keras.applications import ResNet50, MobileNetV2, VGG19, InceptionResNetV2, EfficientNetV2L
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, AveragePooling2D, Dropout, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

# OpenCV - Includes functions for computervision. Used for image preprocessing and SIFT implementation.
import cv2

# Used to get float maxvalue
import sys

# Used for file system operations
import os

# Used to write csv datasets
import csv

# Used to manipulate tabular data (dataframes)
import pandas

# Used to save models
import pickle as pkl
import matplotlib.patches as patches

import tensorflow as tf
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

#### (0b) Fonctions

##### À faire:
Avez-vous créé des fonctions? Si oui, vous devez les mettre ici, avec une description.

In [ ]:
#Basé sur TM_SQDIFF dans opencv (décrit ici: https://docs.opencv.org/3.4/de/da9/tutorial_template_matching.html)
def img_sqdiff(img1, img2):
  return np.sum(np.square(img1 - img2))

def classify_template_match(templates, img):
  min_diff = sys.maxsize
  min_idx = -1

  for idx, template in enumerate(templates):
    diff = img_sqdiff(template, img)
    if diff < min_diff:
      min_diff = diff
      min_idx = idx

  return min_idx

def classify_template_match_dataset(templates, X):
  return [classify_template_match(templates, img) for img in X]

def extract_lbp(img, nb_points, radius, n_bins):
  lbp = feature.local_binary_pattern(img, nb_points, radius, method='uniform')
  (histogram, _) = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
  return histogram

def extract_lbp_dataset(dataset, nb_points, radius, n_bins):
  return np.array([extract_lbp(img[0], nb_points, radius, n_bins) for img in dataset])

def make_csv_dataset(X, y, usage):
  dataset = []
  for (data, label) in zip(X, y):
    data_str = " ".join([str(x) for x in data.ravel()])
    dataset.append([label, data_str, usage])
  return dataset

def write_FER_csv(output_path, data, data_header):
  with open(output_path, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['emotion', data_header, 'USAGE'])
    writer.writerows(data)

def read_FER_csv(input_path, feature_length):
  data = np.loadtxt(input_path, delimiter=',', dtype=str )
  (Xtrain, ytrain) = split_FER_csv(data, 1, 28710, feature_length)
  (Xtest, ytest) = split_FER_csv(data, 28710, 32299, feature_length)
  (Xval, yval) = split_FER_csv(data, 32299, 35888, feature_length)
  return Xtrain, ytrain, Xtest, ytest, Xval, yval

def split_FER_csv(data, start_index, end_index, feature_length):
  X = np.ones((end_index - start_index, feature_length), float)
  
  for i in range(start_index, end_index):
    X[i-start_index] = data[i,1].split(" ")

  y = data[start_index:end_index, 0].astype(int)

  return (X, y)

def eval_model(model, x_train, y_train, x_val, y_val, x_test, y_test, titre):
  # Classification
  ypred_train = model.predict(x_train)
  print(type(ypred_train[0]))
  if not(isinstance(ypred_train[0], np.int32) or isinstance(ypred_train[0], np.int64)):
    ypred_train = np.argmax(ypred_train, axis=1)
        
  training_accuracy = sklearn.metrics.accuracy_score(y_train, ypred_train, normalize=True, sample_weight=None)
  print("Training accuracy: ")
  print(str(training_accuracy * 100) + '%')
  print("Training error: ")
  print(str((1 - training_accuracy) * 100) + '%')

  ypred_val = model.predict(x_val)
  if not(isinstance(ypred_val[0], np.int32) or isinstance(ypred_val[0], np.int64)):
    ypred_val = np.argmax(ypred_val, axis=1)
    
  validation_accuracy = sklearn.metrics.accuracy_score(y_val, ypred_val, normalize=True, sample_weight=None)
  print("Validation accuracy: ")
  print(str(validation_accuracy * 100) + '%')
  print("Validation error: ")
  print(str((1 - validation_accuracy) * 100) + '%')


  ypred_test = model.predict(x_test)
  if not(isinstance(ypred_test[0], np.int32) or isinstance(ypred_test[0], np.int64)):
    ypred_test = np.argmax(ypred_test, axis=1)
    
  test_accuracy = sklearn.metrics.accuracy_score(y_test, ypred_test, normalize=True, sample_weight=None)
  print("Test accuracy: ")
  print(str(test_accuracy * 100) + '%')
  print("Test error: ")
  print(str((1 - test_accuracy) * 100) + '%')


  train_precision = sklearn.metrics.precision_score(y_train, ypred_train, average='micro')
  validation_precision = sklearn.metrics.precision_score(y_val, ypred_val, average='micro')
  test_precision = sklearn.metrics.precision_score(y_test, ypred_test, average='micro')
  avg_precision = (train_precision + validation_precision + test_precision) / 3
  print("Average precision: " + str(avg_precision * 100) + '%')


  # Confusion matrix
  matrice_confusion = sklearn.metrics.confusion_matrix(y_true=y_test, y_pred=ypred_test, labels=[i for i in range(0, len(labels))])
  disp = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix=matrice_confusion, display_labels=labels)
  disp.plot()
  plt.title(f'Matrice de confusion - {titre}')
  plt.show()


def extract_SIFT(images, k):
  (sift, descriptors) = extract_SIFT_features(images)
  kmeans = clustering(descriptors, k)
  sift_histograms = create_histograms(sift, kmeans, images, k)
  return sift_histograms
  

def extract_SIFT_features(images):
    sift = cv2.SIFT_create()
    
    descriptor_list = []
    
    for image in images:
        _, descriptors = sift.detectAndCompute(image, None)
        
        if descriptors is not None:
            for d in descriptors:
                descriptor_list.append(d)
                
    descriptor_list = np.asarray(descriptor_list)
    
    return (sift, descriptor_list.astype('double'))

def clustering(descriptors, k):
    kmeans = MiniBatchKMeans(n_clusters=k, random_state=0).fit(descriptors)
    return kmeans

def create_histograms(sift, kmeans, images, k):
    histogram_list = []
    kmeans = kmeans
    
    for img in images:
        keypoints, descriptors = sift.detectAndCompute(img, None)
        
        histo = np.zeros(k)
        n_keypoints = np.size(keypoints)
        
        if descriptors is not None:
            for d in descriptors:
                idx = kmeans.predict([d])
                histo[idx] += 1/n_keypoints
            
        histogram_list.append(histo)

    return histogram_list

def create_dim_reduction_models(Xtrain, ytrain, n_components, k_features):
    pca = sklearn.decomposition.PCA(n_components=n_components)
    Xtrain = pca.fit_transform(Xtrain)

    k_best = SelectKBest(f_classif, k=k_features)
    k_best.fit_transform(Xtrain, ytrain)

    return (pca, k_best)

def reduce_dimensionality(pca, k_best, Xtrain, Xtest, Xval):
    Xtrain = k_best.transform(pca.transform(Xtrain))
    Xtest = k_best.transform(pca.transform(Xtest))
    Xval = k_best.transform(pca.transform(Xval))
    return (Xtrain, Xtest, Xval)

def plot_dim_reduction(pca, n_components, Xtrain, ytrain):
    Xtrain = SelectKBest(f_classif, k=2).fit_transform(Xtrain, ytrain)
    pixel_pca1 = Xtrain[:,0]
    pixel_pca2 = Xtrain[:,1]

    fig, ax = plt.subplots()
    for label in np.unique(ytrain):
        ix = np.where( ytrain == label)
        ax.scatter(pixel_pca1[ix], pixel_pca2[ix], label = labels[label])
    ax.legend()
    plt.show()

    explained_variance = pca.explained_variance_ratio_
    cum_explained_variance = np.cumsum(explained_variance)
    idx = np.arange(n_components) + 1
    features_explained_variance = pandas.DataFrame([explained_variance, cum_explained_variance], 
                                        index=['explained variance', 'cumulative'], 
                                        columns=idx).T
    mean_explained_variance = features_explained_variance.iloc[:,0].mean() # calculate mean explained variance
        
    print('PCA Overview')
    print(features_explained_variance.head(100))

def eval_jaffe(model, x_test, y_test, titre):
  ypred_test = model.predict(x_test)
  print("Test score: ")
  print(sklearn.metrics.accuracy_score(y_test, ypred_test, normalize=True, sample_weight=None))

  # Confusion matrix
  matrice_confusion = sklearn.metrics.confusion_matrix(y_true=y_test, y_pred=ypred_test, labels=[i for i in range(0, len(labels))])
  disp = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix=matrice_confusion, display_labels=labels)
  disp.plot()
  plt.title(f'Matrice de confusion - {titre}')
  plt.show()

def save_model(model, pkl_name):
    # Save model
    pickle = open(f'Models/{pkl_name}.pkl', 'wb') 
    pkl.dump(model, pickle)
    pickle.close()
    
def convert_to_rgb(Xtrain, Xval, Xtest, width, length):
    Xtrain_rgb = np.zeros((Xtrain.shape[0], width, length, 3))
    Xval_rgb = np.zeros((Xval.shape[0], width, length, 3))
    Xtest_rgb = np.zeros((Xtest.shape[0], width, length, 3))
    
    for i, image in enumerate(Xtrain):
        to_rgb = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
        Xtrain_rgb[i] = to_rgb
        
    for i, image in enumerate(Xval):
        to_rgb = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
        Xval_rgb[i] = to_rgb
        
    for i, image in enumerate(Xtest):
        to_rgb = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
        Xtest_rgb[i] = to_rgb
        
    return Xtrain_rgb, Xval_rgb, Xtest_rgb

def convert_grayscale_to_3_channels(Xtrain, Xval, Xtest, width, length):
    Xtrain_stacked_grayscale = np.zeros((Xtrain.shape[0], width, length, 3))
    Xval_stacked_grayscale = np.zeros((Xval.shape[0], width, length, 3))
    Xtest_stacked_grayscale = np.zeros((Xtest.shape[0], width, length, 3))
    
    for i, image in enumerate(Xtrain):
        stacked_img = np.concatenate((image,)*3, axis=-1)
        Xtrain_stacked_grayscale[i] = stacked_img
    
    for i, image in enumerate(Xval):
        stacked_img = np.concatenate((image,)*3, axis=-1)
        Xval_stacked_grayscale[i] = stacked_img
        
    for i, image in enumerate(Xtest):
        stacked_img = np.concatenate((image,)*3, axis=-1)
        Xtest_stacked_grayscale[i] = stacked_img
        
    return Xtrain_stacked_grayscale, Xval_stacked_grayscale, Xtest_stacked_grayscale

def transfer_train(pre_trained_model, Xtrain, ytrain):
  for layer in pre_trained_model.layers:
      layer.trainable = False

  transfer_trained_model = Sequential()
  transfer_trained_model.add(pre_trained_model)

  # On ajoute les couches pour la classification
  transfer_trained_model.add(Flatten())
  transfer_trained_model.add(Dense(100, activation='relu'))
  transfer_trained_model.add(Dense(len(labels), activation='softmax'))

  # Training avec images greyscale 3 channels (stacked)
  transfer_trained_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  transfer_trained_model.fit(Xtrain, ytrain, epochs=30, batch_size=128, verbose=1, validation_split=0.2)
  return transfer_trained_model

# Partie 1: Explorez les algorithmes de régression
## FG-NET dataset (Facial Aging Estimation)

Dans cette partie vous devez explorer les <b> algorithmes de régression linéaire </b> disponibles dans Scikit-learn, comme régression least square, régression Ridge, régression Lasso et Elastic-Net, descente du gradient stochastique (SGD), etc.

Vous devez comparer la performance de ces algorithmes pour l'ensemble de données FG-NET sur:

1. Le vecteur de pixels (images vectorisées)
2. Vecteur de primitives (reprendre l'algorithme d'extraction de primitives que vous avez utilisées dans le Laboratoire 1, p.ex. LBP). Vous pouvez reprendre les primitives "réduits" que vous avez utilisées dans le Laboratoire 1 (p.ex. vecteur LBP après PCA), si vous pensez qu’ils sont plus performants.

### 1a: Charger le fichier de données

In [ ]:
# Load data
X_fgnet = np.loadtxt('Datasets/fgnet_256x256.csv', delimiter=',', dtype=int )
y_fgnet = np.loadtxt('Datasets/fgnet_labels.csv', delimiter=',', dtype=int )

X_fgnet = X_fgnet.reshape(X_fgnet.shape[0], 256, 256)

### 1b: Visualisation des visages

Vous pouvez visualiser les images en utilisant `plt.imshow`.

Il y a différents types de prétraitement que nous pouvons appliquer à des images dans les ensembles de données pour réduire la variabilité, réduire des bruits, etc.

Voici deux sources pour vous aidez à décider:
- http://eprints.qut.edu.au/92300/1/manuscript_Jhony.pdf
- https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7475805

### À faire:
1. Pensez-vous qu’est nécessaire un prétraitement des images? Si oui, vous pouvez choisir différents algorithmes de prétraitement dans [scikit-image](https://scikit-image.org/docs/stable/api/api.html). Il y a aussi autres types de prétraitement plus généraux dans [scikit-learn](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing).
2. Expliquer et justifier les prétraitements choisis.

#### Code

In [ ]:
face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
X_fgnet_pretreated = []

for image in X_fgnet:
    image = image / np.max(image)
    image = skimage.restoration.denoise_tv_bregman(image)
    image = cv2.resize(image, (48, 48))
    X_fgnet_pretreated.append(image)

X_fgnet = np.array(X_fgnet_pretreated)

#### Résultats et réponses

In [ ]:
# Votre explication/justification

### 1c: Statistiques sur les sujets et étiquettes

##### À faire:
1. Calculer quelques statistiques (# images par sujet, distribution des âges, etc.) que vous jugez importantes sur les données
2. Faire une analyse des résultats et présenter vos conclusions basées sur ces statistiques.

#### Code:

In [ ]:
# Votre code ici

# Code exemple:
# Histogramme des étiquettes
#hist, _ = np.histogram(ytrain, density=False, bins=7, range=(0, 7))

#### Résultats et réponses:

In [ ]:
# Vos résultats ici:

# Code exemple:
# Code exemple
# Plot du histogramme
# import matplotlib.pyplot as plt

### 1d: Créer et évaluer des modèles de régression

##### À faire:
1. Choisir au moins trois (3) algorithmes de régression linéaire disponibles dans Scikit-learn (p.ex. régression least square, régression Ridge, régression Lasso, régression Elastic-Net, descente du gradient stochastique (SGD), etc.)
2. Entraîner et optimiser les paramètres des modèles si nécessaire. Utiliser le protocole <font color=blue> "Leave One Subject Out Cross-Validation" </font> (LOSO).
3. Faire une analyse des résultats et présenter vos conclusions sur les modèles de régression.

| Algorithme            | Paramètres    |  MSE  |  MAE  |
|-----------------------|---------------|-------|-------|
| Regr lineaire         | XXX.XX        |XXX.XX |XXX.XX |
| Regr Ridge            | alpha = 0.1   |123.34 | 10.45 |
| Regr Lasso            | XXX.XX        |XXX.XX |XXX.XX |
| Regr ElasticNet       | XXX.XX        |XXX.XX |XXX.XX |
| ...                   | XXX.XX        |XXX.XX |XXX.XX |
| ...                   | XXX.XX        |XXX.XX |XXX.XX |

#### Code:

In [ ]:
# Votre code ici

#### Résultats et réponses:

In [ ]:
# Vos résultats ici:

# Partie 2: Explorez les algorithmes de classification
## FER dataset ( Facial Expression Recognition )

Vous devez reprendre l'ensemble FER et les primitives que vous avez choisis dans le Laboratoire 1.

Dans cette partie vous devez explorer les algorithmes de classification <b> régression logistique et réseaux de neurones multicouches (MLP) </b>

Vous devez comparer la performance de ces deux algorithmes pour l'ensemble FER sur:
1. Le vecteur de pixels (images vectorisées)
2. Vecteur de primitives (reprendre les primitives ou primitives sélectionnées/transformées du laboratoire 1)

### 2a: Charger le fichier de données

In [ ]:
# Lecture features pixel, SIFT et LBP
(Xtrain, ytrain, Xtest, ytest, Xval, yval) = read_FER_csv('Datasets/fer2013.csv', 2304)
(Xtrain_lbp, ytrain_lbp, Xtest_lbp, ytest_lbp, Xval_lbp, yval_lbp) = read_FER_csv('Datasets/lbp_fer2013.csv', 18)
(Xtrain_sift, ytrain_sift, Xtest_sift, ytest_sift, Xval_sift, yval_sift) = read_FER_csv('Datasets/sift_fer2013.csv', 10)

Xtrain_flatten = Xtrain.reshape( Xtrain.shape[0], 2304 ).astype('uint8')
Xtest_flatten  = Xtest.reshape( Xtest.shape[0], 2304 ).astype('uint8')
Xval_flatten   = Xval.reshape( Xval.shape[0], 2304 ).astype('uint8')

### 2b: Créer et évaluer des modèles de classification (Régression logistique)

##### À faire:

1. Utiliser l'algorithme régression logistique disponible dans [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), pour classifier le vecteur de pixels et le vecteur de primitives du laboratoire 1. Vous pouvez regarder aussi [SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier) et [LogisticRegressionCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html#sklearn.linear_model.LogisticRegressionCV)
2. Entraîner et optimiser les paramètres des modèles.
3. Faire une analyse des résultats et présenter vos conclusions sur le modèle logistique.

#### Code:

In [ ]:
grid_params = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty':['l1', 'l2']
}

lr_classifier = LogisticRegression(solver="saga", max_iter=10000)
grid = GridSearchCV(lr_classifier, grid_params, cv=5, scoring='accuracy')

In [ ]:
# "SIFT"
# Training
sift_grid_search = grid.fit(Xtrain_sift, ytrain)
sift_lr_classifier = sift_grid_search.best_estimator_

print(sift_grid_search.best_params_)

# Save model
save_model(sift_lr_classifier, 'sift_lr_classifier')

In [ ]:
# "pixel"
# Training
pixel_grid_search = grid.fit(Xtrain_flatten, ytrain)
pixel_lr_classifier = pixel_grid_search.best_estimator_

print(pixel_grid_search.best_params_)

# Save model
save_model(pixel_lr_classifier, 'pixel_lr_classifier')

In [ ]:
# "lbp"
# Training
lbp_grid_search = grid.fit(Xtrain_lbp, ytrain)
lbp_lr_classifier = lbp_grid_search.best_estimator_

print(lbp_grid_search.best_params_)

# Save model
save_model(lbp_lr_classifier, 'lbp_lr_classifier')

In [ ]:
eval_model(sift_lr_classifier, Xtrain_sift, ytrain_sift, Xtest_sift, ytest_sift, Xval_sift, yval_sift, 'SIFT logistic regression')
# eval_model(pixel_lr_classifier, Xtrain_flatten, ytrain, Xtest_flatten, ytest, Xval_flatten, yval, 'pixel logistic regression')
eval_model(lbp_lr_classifier, Xtrain_lbp, ytrain, Xtest_lbp, ytest, Xval_lbp, yval, 'LBP logistic regression')

####  Résultats et réponses:

Vos résultats ici:


| Algorithme                    | Paramètres    | Precision | %Erreur App | %Erreur Val | %Erreur Tst | 
|-------------------------------|---------------|-----------|-------------|-------------|-------------|
| Regression logistique - SIFT  | C=10:L2       |   23.79%  |   74.47%    |   76.48%    |   77.68%    |
| Regression logistique - Pixel |               |   xxx.xx  |   xxx.xx    |   XXX.XX    |   XXX.XX    |
| Regression logistique - LBP   | C=10:l2       |   27.14%  |   72.65%    |   72.95%    |   72.97%    |

### 2c: Créer et évaluer des modèles de classification (Réseaux perceptron multi-couche)

##### À faire:

1. Utiliser [Tensorflow et Keras](https://www.tensorflow.org/tutorials/keras/classification) pour construire un réseau de neurones multicouche pour classifier les vecteurs de primitives du laboratoire 1. 
2. Choisir l’architecture appropriée pour chaque vecteur de primitives (nombre et dimension des couches). 
2. Entraîner et optimiser les paramètres des réseaux.
4. Faire une analyse des résultats et présenter vos conclusions sur les réseaux de neurones.

#### Code

In [ ]:
# Convert labels to one hot encoder
ytrain_ohe = to_categorical(ytrain, len(labels))
yval_ohe = to_categorical(yval, len(labels))
ytest_ohe = to_categorical(ytest, len(labels))

In [ ]:
# "SIFT"
sift_input_shape = Xtrain_sift[0].shape

# Create the model
sift_mlp_classifier = Sequential()
sift_mlp_classifier.add(Dense(16, input_shape=sift_input_shape, activation='relu'))
sift_mlp_classifier.add(Dense(len(labels), activation='softmax'))

# Training
sift_mlp_classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
sift_mlp_classifier.fit(Xtrain_sift, ytrain_ohe, epochs=30, batch_size=128, verbose=1, validation_split=0.2)

In [ ]:
# "pixel"
flatten_input_shape = Xtrain_flatten[0].shape

# Create the model
pixel_mlp_classifier = Sequential()
pixel_mlp_classifier.add(Dense(512, input_shape=flatten_input_shape, activation='relu'))
pixel_mlp_classifier.add(Dense(256, activation='relu'))
pixel_mlp_classifier.add(Dense(128, activation='relu'))
pixel_mlp_classifier.add(Dense(64, activation='relu'))
pixel_mlp_classifier.add(Dense(32, activation='relu'))
pixel_mlp_classifier.add(Dense(16, activation='relu'))
pixel_mlp_classifier.add(Dense(len(labels), activation='softmax'))

# Training
pixel_mlp_classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
pixel_mlp_classifier.fit(Xtrain_flatten, ytrain_ohe, epochs=30, batch_size=128, verbose=1, validation_split=0.2)

In [ ]:
# "lbp"
input_shape = Xtrain_lbp[0].shape

# Create the model
lbp_mlp_classifier = Sequential()
lbp_mlp_classifier.add(Dense(64, input_shape=input_shape, activation='relu'))
lbp_mlp_classifier.add(Dense(32, activation='relu'))
lbp_mlp_classifier.add(Dense(16, activation='relu'))
lbp_mlp_classifier.add(Dense(len(labels), activation='softmax'))

# Training
lbp_mlp_classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lbp_mlp_classifier.fit(Xtrain_lbp, ytrain_ohe, epochs=30, batch_size=128, verbose=1, validation_split=0.2)

In [ ]:
eval_model(sift_mlp_classifier, Xtrain_sift, ytrain, Xtest_sift, ytest, Xval_sift, yval, 'SIFT mlp')
eval_model(pixel_mlp_classifier, Xtrain_flatten, ytrain, Xtest_flatten, ytest, Xval_flatten, yval, 'pixel mlp')
eval_model(lbp_mlp_classifier, Xtrain_lbp, ytrain, Xtest_lbp, ytest, Xval_lbp, yval, 'LBP mlp')

#### Résultats et réponses:
Vos résultats ici

| Algorithme       | Paramètres             | Precision | %Erreur App | %Erreur Val | %Erreur Tst | 
|------------------|------------------------|-----------|-------------|-------------|-------------|
| MLP SIFT         | 16:7                   |   24.45%  |   74.54%    |   75.26%    |   76.76%    |
| MLP Pixel        | 512:256:128:64:32:16:7 |   24.87%  |   74.85%    |   75.03%    |   75.51%    |
| MLP LBP          | 64:32:16:7             |   26.77%  |   73.58%    |   72.58%    |   73.53%    |

# Partie 3: Explorez l'apprentissage de la représentation et les réseaux neuronaux profonds
## (FER dataset et FG-NET dataset)


##### À faire :
1. Utiliser [Tensorflow et Keras](https://www.tensorflow.org/tutorials/keras/classification) pour construire un réseau de neurones convolutif pour apprendre une représentation directement des images de visage aussi que les discriminantes. 
2. Choisir l’architecture appropriée : nombre de couches convolutifs (CL), dimension des filtres, “stride”, nombre de couches entièrement connectées (FC) et la dimension des couches). 
3. Entraîner et optimiser les paramètres du réseau.
4. Évaluer la pertinence d’utiliser "data augmentation" pour améliorer la généralisation.
5. Récupérer une des architectures CNN pré-entraînes disponibles dans Keras pour faire une “transfert de connaissance (transfer learning). Faire un "fine-tuning" du modèle choisi sur FER.    
7. Choisir le modèle qu’a donné de meilleurs résultats sur FER et adapter ce modèle pour faire la régression sur FG-NET.     
8. Faire une analyse des résultats et présenter vos conclusions sur les réseaux de neurones.

### 3a: Code CNN (FER)
    
##### À faire:
1. Utiliser [Tensorflow et Keras](https://www.tensorflow.org/tutorials/keras/classification) pour construire un réseau de neurones convolutif pour apprendre une représentation directement des images de visage aussi que les discriminantes. 
2. Choisir l’architecture appropriée: nombre de couches convolutifs (CL), dimension des filtres, "stride”, nombre de couches entièrement connectées (FC) et la dimension des couches). 
3. Entraîner et optimiser les paramètres du réseau.
4. Évaluer la pertinence d’utiliser "data augmentation" pour améliorer la généralisation.

In [ ]:
# Lecture features pixel, SIFT et LBP
(Xtrain, ytrain, Xtest, ytest, Xval, yval) = read_FER_csv('Datasets/fer2013.csv', 2304)

Xtrain = Xtrain.reshape( Xtrain.shape[0], 48, 48, 1 ).astype('uint8')
Xtest  = Xtest.reshape( Xtest.shape[0], 48, 48, 1 ).astype('uint8')
Xval   = Xval.reshape( Xval.shape[0], 48, 48, 1 ).astype('uint8')

ytrain_ohe = to_categorical(ytrain, len(labels))
yval_ohe = to_categorical(yval, len(labels))
ytest_ohe = to_categorical(ytest, len(labels))

In [ ]:
#Architecture adaptée à partir de LE: https://towardsdatascience.com/understanding-and-implementing-lenet-5-cnn-architecture-deep-learning-a2d531ebc342
#"pixel"
# Create the model
pixel_cnn = Sequential()

pixel_cnn.add(Conv2D(16, (5, 5), padding='same', input_shape=Xtrain[0].shape, activation='relu'))
pixel_cnn.add(AveragePooling2D((2, 2), strides=2))

pixel_cnn.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
pixel_cnn.add(AveragePooling2D((2, 2), strides=2))

pixel_cnn.add(Flatten()),
pixel_cnn.add(Dense(120, activation='relu'))
pixel_cnn.add(Dropout(0.3))
pixel_cnn.add(Dense(84, activation='relu'))
pixel_cnn.add(Dropout(0.3))
pixel_cnn.add(Dense(ytrain_ohe[0].shape[0], activation='softmax'))

# Training
pixel_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
pixel_cnn.fit(Xtrain, ytrain_ohe, epochs=30, batch_size=128, verbose=1, validation_split=0.2)

In [ ]:
eval_model(pixel_cnn, Xtrain, ytrain, Xtest, ytest, Xval, yval, 'pixel cnn')

#### Résultats et réponses:
Vos résultats ici

Exemple

| Ensemble | My CNN   |
|----------|----------|
| App      | 74.83%   |
| Val      | 46.75%   |
| Test     | 47.92%   |
| Moyenne  | 56.50%   |

 
Q1: Nous avons choisi l'algorithme d'arbre de décision parce que...

Q2: Les taux de classification indiquent que le modèle...

Q3: La performance avec le primitive...

### 3b: Code CNN Pré-entraîné (FER)

##### À faire:
1. Récupérer une des architectures [CNN pré-entraînes disponibles dans Keras](https://keras.io/api/applications/) pour faire un "transfert de connaissance" (transfer learning). Faire un "fine-tuning" du modèle choisi sur FER.    
2. Entraîner et optimiser les paramètres du réseau.
3. Évaluer la pertinence d’utiliser "data augmentation" pour améliorer la généralisation.

In [ ]:
Xtrain_rgb, Xval_rgb, Xtest_rgb = convert_to_rgb(Xtrain, Xval, Xtest, 48, 48)
Xtrain_stacked_grayscale, Xval_stacked_grayscale, Xtest_stacked_grayscale = convert_grayscale_to_3_channels(Xtrain, Xval, Xtest, 48, 48)

In [ ]:

pretrained_resnet50 = ResNet50(include_top=False,
                               input_shape=(48,48,3),
                               pooling='avg',classes=len(labels),
                               weights='imagenet')

resnet50 = transfer_train(pretrained_resnet50, Xtrain_rgb, ytrain_ohe)
resnet50.save("Models/resnet50")

In [ ]:
pretrained_mobilenet = MobileNetV2(include_top=False,
                               input_shape=(48,48,3),
                               pooling='avg',classes=len(labels),
                               weights='imagenet')

mobilenet = transfer_train(pretrained_mobilenet, Xtrain_rgb, ytrain_ohe)
mobilenet.save("Models/mobilenet")

In [ ]:
pretrained_vgg19 = VGG19(include_top=False,
                               input_shape=(48,48,3),
                               pooling='avg',classes=len(labels),
                               weights='imagenet')

vgg19 = transfer_train(pretrained_vgg19, Xtrain_rgb, ytrain_ohe)
vgg19.save('Models/vgg19')

In [ ]:
pretrained_EfficientNetV2L= EfficientNetV2L(include_top=False,
                               input_shape=(48,48,3),
                               pooling='avg',classes=len(labels),
                               weights='imagenet')

efficientNetV2L = transfer_train(pretrained_vgg19, Xtrain_rgb, ytrain_ohe)
efficientNetV2L.save('Models/efficientNet')

#### Résultats et réponses:

In [ ]:
resnet50 = keras.models.load_model("Models/resnet50")
mobilenet = keras.models.load_model("Models/mobilenet")
vgg19 = keras.models.load_model('Models/vgg19')
efficientNetV2L = keras.models.load_model('Models/efficientNet')

eval_model(resnet50, Xtrain_stacked_grayscale, ytrain, Xtest_stacked_grayscale, ytest, Xval_stacked_grayscale, yval, 'pixel ResNet50')
eval_model(mobilenet, Xtrain_stacked_grayscale, ytrain, Xtest_stacked_grayscale, ytest, Xval_stacked_grayscale, yval, 'pixel MobileNet')
eval_model(vgg19, Xtrain_stacked_grayscale, ytrain, Xtest_stacked_grayscale, ytest, Xval_stacked_grayscale, yval, 'pixel ResNet50')
eval_model(efficientNetV2L, Xtrain_stacked_grayscale, ytrain, Xtest_stacked_grayscale, ytest, Xval_stacked_grayscale, yval, 'pixel ResNet50')

Vos résultats ici

##### Exemple:
| Ensemble | My CNN   | Resnet50   | MobileNet | VGG 19 | EfficientNetV2L |
|----------|----------|------------|-----------|--------|-----------------|
| App      | 74.83%   |     80.32% |    47.29% | 62.84% |          65.58% |
| Val      | 46.75%   |     43.69% |    30.29% | 37.16% |          39.45% |
| Test     | 47.92%   |     45.33% |    29.73% | 39.12% |          39.98% |
| Moyenne  | 56.50%   |     56.45% |    35.77% | 47.06% |          48.34% |

### 3c: Code CNN Pré-entraîné (FG-NET):

##### À faire:
1. Choisir le modèle \ l’architecture qu’a donné de meilleurs résultats sur FER pour faire un “transfert de connaissance (transfer learning).
2. Adapter ce modèle pour FG-NET (couche de sortie classification (7 unités avec softmax) -> régression (1 unité lineaire)).   
3. Entraîner ("fine-tuning" du modèle choisi sur FG-NET) et optimiser les paramètres du réseau pour la régression.
4. Évaluer la pertinence d’utiliser "data augmentation" pour améliorer la généralisation.
5. Faire une analyse des résultats et présenter vos conclusions sur les réseaux de neurones.

In [ ]:
#"pixel"
# Create the model
pixel_cnn = Sequential()

pixel_cnn.add(Conv2D(16, (5, 5), padding='same', input_shape=Xtrain[0].shape, activation='relu'))
pixel_cnn.add(AveragePooling2D((2, 2), strides=2))

pixel_cnn.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
pixel_cnn.add(AveragePooling2D((2, 2), strides=2))

pixel_cnn.add(Flatten()),
pixel_cnn.add(Dense(120, activation='relu'))
pixel_cnn.add(Dropout(0.3))
pixel_cnn.add(Dense(84, activation='relu'))
pixel_cnn.add(Dropout(0.3))
pixel_cnn.add(Dense(42, activation='relu'))
pixel_cnn.add(Dense(1, activation='linear'))

# Training
pixel_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
pixel_cnn.fit(Xtrain, ytrain_ohe, epochs=30, batch_size=128, verbose=1, validation_split=0.2)

#### Résultats et réponses:

Vos résultats ici

##### Exemple:
| Ensemble | My CNN   | VGG19 CNN           |                               
|----------|----------|---------------------|
| App      | 99,67%   |     XX,XX%          |                   
| Val      | 89,77%   |     XX,XX%          |                             
| Test     | 77,99%   |     XX,XX%          |       


Q1: Nous avons choisi l'algorithme d'arbre de décision parce que...

Q2: Les taux de classification indiquent que le modèle...

Q3: La performance avec le primitive...

## Partie Final: Conclusion

##### À faire:
1. Résumer et comparer les principaux résultats obtenus pour la classification (FER) et régression (FG-NET).
2. Faire une analyse des résultats obtenus et présenter vos conclusions sur les différents modèles que vous avez entraînés (classification et régression).
3. Comparer les résultats obtenus sur FER avec les résultats du laboratoire 1 (vecteur de pixels, vecteur de primitives). Avez-vous observé une amélioration? Commenter sur les temps d'apprentissage, complexité spatiale et temporelle, etc.

#### Résultats et réponses:
Vos résultats ici

| Algorithme            | MSE           |
|-----------------------|---------------|
| Regr lineaire         | XXX.XX        |
| Regr Ridge            | XXX.XX        |
| Regr Lasso            | XXX.XX        |
| Regr ElasticNet       | XXX.XX        |
| ...                   | XXX.XX        |
| ...                   | XXX.XX        |